In [4]:
import pandas as pd,re, pickle, scipy
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import *
import numpy as np
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.metrics import *
from sklearn.kernel_ridge import *
from sklearn import tree
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.model_selection import *
from sklearn.linear_model import *
from collections import defaultdict

from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from url_utils import URLUtils

from encoder import Encoder
import sys
reload(sys)
pca = PCA()#n_components = 'mle', svd_solver = 'full')
sys.setdefaultencoding("utf-8")
def parse_dates(date):
    parts=date.split(" ")
    date_parts = parts[0].split("-")
    time_parts = parts[1].split(':')
    new_date =date_parts[2]+date_parts[1]+date_parts[0]+time_parts[0]+time_parts[1]
    return int(new_date)

In [3]:

df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/email_campaign/train_HFxi8kT/train.csv')

df['send_date'] = map(parse_dates,df['send_date'])

In [3]:

# cdf = pd.read_csv('/Users/bjayaram/Documents/Kaggle/email_campaign/train_HFxi8kT/campaign_data.csv')
# cdf['text']=cdf['communication_type']+" "+get_text(cdf['email_url'])

# cdf['url_text'] = cdf['email_url'].map(URLUtils.parse_urls)
# pickle.dump(cdf,open('/Users/bjayaram/Documents/Kaggle/email_campaign/cdf.p','w'))

In [13]:
def encode_data(x_train, numerical_columns, d, enc_create=True):
        print "Numerical columns",str(numerical_columns)
        if numerical_columns is not None:
            cat_cols = x_train.drop(numerical_columns, axis=1).fillna('NA')
        else:
            cat_cols = x_train

        label_enc_map = {}

        if enc_create:
            enc = ()
            cat_data = cat_cols.apply(lambda x: d[x.name].fit_transform(x))
        else:
            cat_data = cat_cols.apply(lambda x: d[x.name].transform(x))
        
        if numerical_columns is not None:
            df = pd.concat([x_train[numerical_columns].fillna(0), cat_data], axis=1)
        else:
            df = cat_data
        return df.values


In [15]:


cdf = pickle.load(open('/Users/bjayaram/Documents/Kaggle/email_campaign/cdf.p'))
cdf['text'] = cdf['communication_type']+' '+cdf['subject'] +' '+cdf['email_body']#+' '+cdf['url_text']
d=defaultdict(LE)
print cdf['communication_type']
cdf['comm'] = encode_data(cdf[['communication_type']], None, d)


vec = TfidfVectorizer()
vec_data = pca.fit_transform(vec.fit_transform(cdf['text'].values).todense())

pickle.dump(csr_matrix(vec_data), open('/Users/bjayaram/Documents/Kaggle/email_campaign/vec_data.p','w'))


In [16]:
vec_data = pickle.load(open('/Users/bjayaram/Documents/Kaggle/email_campaign/vec_data.p'))
print cdf['comm']
data = vec_data.todense()
type(data), type(vec_data)

(numpy.matrixlib.defmatrix.matrix, scipy.sparse.csr.csr_matrix)

In [13]:
m,n = data.shape
m,n

In [17]:

campaign_data = {}
campaign_ids = cdf['campaign_id'].values
campaign_comm_type = cdf['comm'].values
print campaign_ids, len(campaign_ids)
for i in range(len(campaign_ids)):
    j=campaign_ids[i]
    campaign_data[j] = [campaign_comm_type[i]]+list(np.asarray(np.asarray(data[i])[0]))
    i,len(campaign_data[j])


In [16]:
data_cols = ['user_id','send_date']
label_cols = ['is_click']
campaign_features = np.asarray(map(lambda x: campaign_data[x],df['campaign_id']))
click_features = df[data_cols].values

campaign_features
click_features

array([[       14051, 201709011955],
       [      134438, 201711021253],
       [      181789, 201707241515],
       ..., 
       [      235406, 201707260937],
       [      183424, 201711062253],
       [       89219, 201712012020]])

In [17]:

X = np.concatenate([click_features, campaign_features], axis=1)

Y = df['is_click'].values
X,Y

(array([[  1.40510000e+04,   2.01709012e+11,   3.00000000e+00, ...,
          -1.17196805e-04,   1.13485962e-16,   4.00420661e-19],
        [  1.34438000e+05,   2.01711021e+11,   3.00000000e+00, ...,
           2.88750758e-04,   1.13485962e-16,   4.00420661e-19],
        [  1.81789000e+05,   2.01707242e+11,   4.00000000e+00, ...,
           1.99088149e-04,   1.13485962e-16,   4.00420661e-19],
        ..., 
        [  2.35406000e+05,   2.01707261e+11,   4.00000000e+00, ...,
           1.99088149e-04,   1.13485962e-16,   4.00420661e-19],
        [  1.83424000e+05,   2.01711062e+11,   0.00000000e+00, ...,
           3.41047971e-05,   1.13485962e-16,   4.00420661e-19],
        [  8.92190000e+04,   2.01712012e+11,   3.00000000e+00, ...,
          -9.97446609e-05,   1.13485962e-16,   4.00420661e-19]]),
 array([0, 0, 0, ..., 1, 0, 0]))

In [18]:

mm = MinMaxScaler()
X = csr_matrix(mm.fit_transform(X))

X.shape, Y.shape
dataset = (X,Y)  
pickle.dump(dataset,open('/Users/bjayaram/Documents/Kaggle/email_campaign/data_with_subject_body.p','w'))
pickle.dump(mm,open('/Users/bjayaram/Documents/Kaggle/email_campaign/mmscaler.p','w'))



In [2]:
dataset = pickle.load(open('/Users/bjayaram/Documents/Kaggle/email_campaign/data_with_subject_body.p'))
mm =pickle.load(open('/Users/bjayaram/Documents/Kaggle/email_campaign/mmscaler.p'))
X=dataset[0]
Y=dataset[1]
X,Y
#  


(<1023191x55 sparse matrix of type '<type 'numpy.float64'>'
 	with 54034237 stored elements in Compressed Sparse Row format>,
 array([0, 0, 0, ..., 1, 0, 0]))

In [19]:

# chi = SelectKBest(score_func=chi2, k=500)
# X = pca.fit_transform(X.todense(),Y)
# X = chi.fit_transform(X,Y)
# 
# X.shape, Y.shape
# 
# 'Data loaded'

In [37]:


# model = GridSearchCV(LinearSVC(), cv = 5, n_jobs=24, param_grid={'C':range(1,100,20),'class_weight':['balanced']})

# model = LinearSVC(class_weight='balanced')
# model = RandomForestClassifier(n_estimators= 1000, n_jobs=-1)
# model = LogisticRegressionCV(Cs = range(1,100,10))
# model = GridSearchCV(GradientBoostingClassifier(), n_jobs=7, verbose=5, param_grid={'learning_rate':[0.1],'class_weight':['balanced']})
# model = GaussianNB(); X = X.todense()
# print cross_val_score(model, X,Y,scoring='f1', n_jobs=-1)
kf = StratifiedKFold()

model = GridSearchCV(LinearSVC(), n_jobs=-1, verbose=5, param_grid={'class_weight':['balanced']})
model.fit(X,Y)
# 
"Grid best scores = ",model.best_score_
"Grid all scores = ",model.best_params_


('Grid all scores = ', {'class_weight': 'balanced'})

In [38]:
df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/email_campaign/test_BDIfz5B.csv')

df['send_date'] = map(parse_dates,df['send_date'])


In [38]:
data_cols = ['user_id','send_date']
click_features = df[data_cols].values
campaign_features = np.asarray(map(lambda x: campaign_data[x],df['campaign_id']))
X_test = np.concatenate([click_features, campaign_features], axis=1)
X_test=csr_matrix(mm.transform(X_test))

pickle.dump(X_test,open('/Users/bjayaram/Documents/Kaggle/email_campaign/X_test.p','w'))

In [23]:

X_test=pickle.load(open('/Users/bjayaram/Documents/Kaggle/email_campaign/X_test.p'))

# Y_test = df['is_click'].values
# X_test = chi.transform(X_test)


Y_test = model.predict(X_test.todense())
df['is_click']=Y_test


In [24]:
results = df[['id', 'is_click']]
results.to_csv('/Users/bjayaram/Documents/Kaggle/email_campaign/results.csv', index=None)